# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-b747603b13-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Tv series Without a Trace explorative search") 


Consider the following exploratory scenario:


> we are interested in the TV series "Without a Trace" and we want to investigate the main aspects related to the actors and directors involved in the production, know the numerber of seasons and check what are the episodes which got the higher success/impact.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P4969`    | derivative work      | predicate |
| `wd:Q826477` | Without a Trace    | node |
| `wd:Q733960` | Cold Case       | node |



Also consider

```
wd:Q826477 ?p ?obj .
```

is the BGP to retrieve all **properties of Without a Trace**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should

1. Identify the BGP for television series

2. Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

3. Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

4. Check who is the actor who acted in more films while working on "Without a Trace" (the result set must be a list of triples actor/actress IRI, label and #films).

5. Compare Without a Trace with the tv series "Cold Case" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

6. Return the actors who are members of the cast of Without a Trace have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

7. Consider the actors who are members of the cast of Cold Case. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

## Task 1
Identify the BGP for television series

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q826477 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('name', 'list of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P2047'), ('name', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P2

In [5]:
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # using instance of property
   wd:Q826477 wdt:P31 ?obj .
   
   # get the label
   #?p sc:name ?name.
   ?obj sc:name ?objname .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5398426'), ('objname', 'television series')]
1


Final query for this task

In [6]:
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # using instance of property
   wd:Q826477 wdt:P31 ?obj .
   
   # get the label
   #?p sc:name ?name.
   ?obj sc:name ?objname .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5398426'), ('objname', 'television series')]
1


## Task 2
Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

In [ ]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # Find related characteristics 
   wd:Q826477 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   FILTER (REGEX (?name, ".*season.*") || REGEX(?name, ".*episode.*"))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('name', 'list of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('name', 'number of seasons')]
3


In [24]:
#[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('name', 'number of seasons')]
#[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]

queryString = """
SELECT DISTINCT ?seasons ?episodes 
WHERE {

   #Returning number of seasons
   wd:Q826477 wdt:P2437 ?seasons .
   #Returning number of episodes
   wd:Q826477 wdt:P1113 ?episodes .
   
   # get the label
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('seasons', '7'), ('episodes', '160')]
1


In [2]:
#[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('name', 'list of episodes')]

queryString = """
SELECT DISTINCT ?episodes ?episodeName
WHERE {

   #Returning list of episodes
   wd:Q826477 wdt:P1811 ?episodes .
   
   # get the label
   ?episodes sc:name ?episodeName
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('episodes', 'http://www.wikidata.org/entity/Q282789'), ('episodeName', 'list of Without a Trace episodes')]
1


In [3]:
#[('episodes', 'http://www.wikidata.org/entity/Q282789'), ('episodeName', 'list of Without a Trace episodes')]

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {

   #
   wd:Q282789 ?p ?o .
   
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pname', 'is a list of'), ('o', 'http://www.wikidata.org/entity/Q1983062'), ('oname', 'episode')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q13406463'), ('oname', 'Wikimedia list article')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pname', 'is a list of'), ('o', 'http://www.wikidata.org/entity/Q21191270'), ('oname', 'television series episode')]
3


In [8]:
#[('episodes', 'http://www.wikidata.org/entity/Q282789'), ('episodeName', 'list of Without a Trace episodes')]
# [('p', 'http://www.wikidata.org/prop/direct/P360'), ('pname', 'is a list of')

queryString = """
SELECT DISTINCT ?list ?listName
WHERE {

   #
   #?list wdt:P360 wd:Q282789 .
   wd:Q282789 wdt:P360 ?list .
   
   # get the label
   #?p sc:name ?pname.
   #?o sc:name ?oname .
   ?list sc:name ?listName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('list', 'http://www.wikidata.org/entity/Q1983062'), ('listName', 'episode')]
[('list', 'http://www.wikidata.org/entity/Q21191270'), ('listName', 'television series episode')]
2


In [10]:
#[('episodes', 'http://www.wikidata.org/entity/Q282789'), ('episodeName', 'list of Without a Trace episodes')]

queryString = """
SELECT DISTINCT ?episodes ?episodeName
WHERE {

   #
   wd:Q282789 wdt:P31 ?episodes .
   
   # get the label
   ?episodes sc:name ?episodeName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('episodes', 'http://www.wikidata.org/entity/Q13406463'), ('episodeName', 'Wikimedia list article')]
1


In [12]:
#From Task 1: [('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]

queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {

   # using has part property
   wd:Q826477 wdt:P527 ?obj .
   
   # get the label
   ?obj sc:name ?objname.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('obj', 'http://www.wikidata.org/entity/Q1120248'), ('objname', 'Without a Trace, season 1')]
[('obj', 'http://www.wikidata.org/entity/Q3729810'), ('objname', 'Without a Trace, season 4')]
[('obj', 'http://www.wikidata.org/entity/Q3729811'), ('objname', 'Without a Trace, season 5')]
[('obj', 'http://www.wikidata.org/entity/Q3729812'), ('objname', 'Without a Trace, season 2')]
[('obj', 'http://www.wikidata.org/entity/Q3729815'), ('objname', 'Without a Trace, season 6')]
[('obj', 'http://www.wikidata.org/entity/Q3729816'), ('objname', 'Without a Trace, season 7')]
[('obj', 'http://www.wikidata.org/entity/Q3729817'), ('objname', 'Without a Trace, season 3')]
7


Final query for this task

In [13]:
#[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]

queryString = """
SELECT DISTINCT ?season ?seasonName ?episode
WHERE {

   # using has part property
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P1113 ?episode .
   
   # get the label
   ?season sc:name ?seasonName.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('season', 'http://www.wikidata.org/entity/Q1120248'), ('seasonName', 'Without a Trace, season 1'), ('episode', '23')]
[('season', 'http://www.wikidata.org/entity/Q3729810'), ('seasonName', 'Without a Trace, season 4'), ('episode', '24')]
[('season', 'http://www.wikidata.org/entity/Q3729811'), ('seasonName', 'Without a Trace, season 5'), ('episode', '24')]
[('season', 'http://www.wikidata.org/entity/Q3729812'), ('seasonName', 'Without a Trace, season 2'), ('episode', '24')]
[('season', 'http://www.wikidata.org/entity/Q3729815'), ('seasonName', 'Without a Trace, season 6'), ('episode', '18')]
[('season', 'http://www.wikidata.org/entity/Q3729816'), ('seasonName', 'Without a Trace, season 7'), ('episode', '24')]
[('season', 'http://www.wikidata.org/entity/Q3729817'), ('seasonName', 'Without a Trace, season 3'), ('episode', '23')]
7


## Task 3
Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

In [15]:
# From Task 1: [('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]

queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {

   # using has part property
   wd:Q826477 wdt:P161 ?obj .
   
   # get the label
   ?obj sc:name ?objname.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)




Results
[('obj', 'http://www.wikidata.org/entity/Q308124'), ('objname', 'Anthony LaPaglia')]
[('obj', 'http://www.wikidata.org/entity/Q235075'), ('objname', 'Poppy Montgomery')]
2


In [39]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {

   # using has part property
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   #Finding properties of an episode
   ?episode ?p ?o .
   
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pname', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pname', 'original language of film or TV show')]
[('p', 'http://www.wikidata.org/prop/direct/P4908'), ('pname', 'season')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pname', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('pname', 'screenwriter')]
11


In [40]:
# From Results: [('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]

queryString = """
SELECT DISTINCT ?seasonName ?episodeName ?castName
WHERE {

   # using has part property
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   # get the label
   ?season sc:name ?seasonName.
   ?episode sc:name ?episodeName .
   ?cast sc:name ?castName .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('seasonName', 'Without a Trace, season 1'), ('episodeName', 'Birthday Boy'), ('castName', 'David Paymer')]
[('seasonName', 'Without a Trace, season 1'), ('episodeName', 'Pilot'), ('castName', 'Zach Grenier')]
[('seasonName', 'Without a Trace, season 7'), ('episodeName', 'Devotion'), ('castName', 'Lucille Soong')]
[('seasonName', 'Without a Trace, season 1'), ('episodeName', 'Maple Street'), ('castName', 'Marianne Jean-Baptiste')]
[('seasonName', 'Without a Trace, season 1'), ('episodeName', 'Underground Railroad'), ('castName', 'Marianne Jean-Baptiste')]
[('seasonName', 'Without a Trace, season 1'), ('episodeName', 'Hang On to Me'), ('castName', 'Marianne Jean-Baptiste')]
[('seasonName', 'Without a Trace, season 2'), ('episodeName', 'Our Sons and Daughters'), ('castName', 'Marianne Jean-Baptiste')]
[('seasonName', 'Without a Trace, season 2'), ('episodeName', 'A Tree Falls'), ('castName', 'Marianne Jean-Baptiste')]
[('seasonName', 'Without a Trace, season 2'), ('episodeName',

Final query for this task

In [3]:
# From Task 1: [('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
#  [('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]

queryString = """
SELECT DISTINCT ?cast ?castName (COUNT(?episode) as ?episodeCount )
WHERE {

   # using has part property
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   # get the label
   ?season sc:name ?seasonName.
   ?episode sc:name ?episodeName .
   ?cast sc:name ?castName .
}
GROUP BY (?cast) (?castName)
ORDER BY DESC (?episodeCount)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q308124'), ('castName', 'Anthony LaPaglia'), ('episodeCount', '160')]
[('cast', 'http://www.wikidata.org/entity/Q235075'), ('castName', 'Poppy Montgomery'), ('episodeCount', '160')]
[('cast', 'http://www.wikidata.org/entity/Q503040'), ('castName', 'Eric Close'), ('episodeCount', '160')]
[('cast', 'http://www.wikidata.org/entity/Q551608'), ('castName', 'Enrique Murciano'), ('episodeCount', '160')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('castName', 'Marianne Jean-Baptiste'), ('episodeCount', '160')]
[('cast', 'http://www.wikidata.org/entity/Q459638'), ('castName', 'Stacy Edwards'), ('episodeCount', '1')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('castName', 'Charles S. Dutton'), ('episodeCount', '1')]
[('cast', 'http://www.wikidata.org/entity/Q5363239'), ('castName', 'Elizabeth Morehead'), ('episodeCount', '1')]
[('cast', 'http://www.wikidata.org/entity/Q64684485'), ('castName', 'Caris Vujcec'), ('episodeCount',

## Task 4
Check who is the actor who acted in more films while working on "Without a Trace" (the result set must be a list of triples actor/actress IRI, label and #films).

In [202]:
# all cast members of the Without a trace

queryString = """
SELECT DISTINCT ?cast ?castName
WHERE {

   # using has part property to find casts
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   

   # get the label
   #?season sc:name ?seasonName.
   #?episode sc:name ?episodeName .
   ?cast sc:name ?castName .

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('cast', 'http://www.wikidata.org/entity/Q373989'), ('castName', 'David Paymer')]
[('cast', 'http://www.wikidata.org/entity/Q139341'), ('castName', 'Zach Grenier')]
[('cast', 'http://www.wikidata.org/entity/Q3265830'), ('castName', 'Lucille Soong')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('castName', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity/Q238501'), ('castName', 'Alex Veadov')]
[('cast', 'http://www.wikidata.org/entity/Q308124'), ('castName', 'Anthony LaPaglia')]
[('cast', 'http://www.wikidata.org/entity/Q1721796'), ('castName', 'Kaitlin Doubleday')]
[('cast', 'http://www.wikidata.org/entity/Q355163'), ('castName', 'Bruce Davison')]
[('cast', 'http://www.wikidata.org/entity/Q22704603'), ('castName', 'Lily Knight')]
[('cast', 'http://www.wikidata.org/entity/Q445325'), ('castName', 'Megan Gallagher')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('castName', 'Charles S. Dutton')]
[('cast', 'http://www.wikidata.org/entity/Q11256

In [203]:
#From Task 1: [('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
#[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]

queryString = """
SELECT DISTINCT ?start ?end
WHERE {

   # using start time and end time property to bound period
   wd:Q826477 wdt:P580 ?start .
   wd:Q826477 wdt:P582 ?end .
   

   # get the label

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('start', '2002-09-26T00:00:00Z'), ('end', '2009-05-19T00:00:00Z')]
1


In [5]:
# Trying to find release date property for films
#[('obj', 'http://www.wikidata.org/entity/Q5398426'), ('objname', 'television series')]

queryString = """
SELECT DISTINCT  ?p ?pname ?o ?oname 
WHERE {

   wd:Q5398426 wdt:P279 ?obj.
   ?obj ?p ?o .
   

   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname .
   
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of'), ('o', 'http://www.wikidata.org/entity/Q15704878'), ('oname', 'television programming')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q20937557'), ('oname', 'series')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P580'), ('oname', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P582'), ('oname', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q5616006'), ('oname', 'Template:Infobox television')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q1138524'), ('oname', 'Paulys Realenzyklopädie der

In [10]:
#From results: [('p', 'http://www.wikidata.org/prop/direct/P279'),
#('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q20937557'), ('oname', 'series')]

queryString = """
SELECT DISTINCT  ?s ?sname 
WHERE {

   #Trying to find class Film
   ?s wdt:P279 wd:Q20937557.
  
   

   # get the label
   ?s sc:name ?sname .
   
   
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)




Results
[('s', 'http://www.wikidata.org/entity/Q332225'), ('sname', 'endgame')]
[('s', 'http://www.wikidata.org/entity/Q98391050'), ('sname', 'nuclear tests series')]
[('s', 'http://www.wikidata.org/entity/Q31836768'), ('sname', 'model series')]
[('s', 'http://www.wikidata.org/entity/Q133250'), ('sname', 'sequence')]
[('s', 'http://www.wikidata.org/entity/Q3249551'), ('sname', 'process')]
[('s', 'http://www.wikidata.org/entity/Q11471'), ('sname', 'time')]
[('s', 'http://www.wikidata.org/entity/Q52905625'), ('sname', 'numismatic serie')]
[('s', 'http://www.wikidata.org/entity/Q12538685'), ('sname', 'roadshow')]
[('s', 'http://www.wikidata.org/entity/Q11424'), ('sname', 'film')]
[('s', 'http://www.wikidata.org/entity/Q15142889'), ('sname', 'weapon family')]
[('s', 'http://www.wikidata.org/entity/Q1914636'), ('sname', 'activity')]
[('s', 'http://www.wikidata.org/entity/Q1326471'), ('sname', 'chord progression')]
[('s', 'http://www.wikidata.org/entity/Q357384'), ('sname', 'cascade')]
[('s'

In [17]:
#From results: [('s', 'http://www.wikidata.org/entity/Q11424'), ('sname', 'film')]

queryString = """
SELECT DISTINCT  ?p ?pname ?o ?oname
WHERE {

   #trying to find property related to release date
   wd:Q11424 ?p ?o.
  
   

   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname .
   
   FILTER (REGEX (?pname , ".*date.*"))
   
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [18]:
#From results: [('s', 'http://www.wikidata.org/entity/Q11424'), ('sname', 'film')]

queryString = """
SELECT DISTINCT  ?p ?pname ?o ?oname
WHERE {

   #trying to find property related to release date
   wd:Q11424 ?p ?o.
  
   

   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname .
   
   FILTER (REGEX (?oname , ".*date.*"))
   
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P577'), ('oname', 'publication date')]
1


In [19]:
#From results: [('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'),
#('o', 'http://www.wikidata.org/entity/P577'), ('oname', 'publication date')]

#All films that actors of the Without a trace played during playing this film with their publication date

queryString = """
SELECT DISTINCT ?film ?filmName ?date
WHERE {

   # using has part property to find casts
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   ?film wdt:P161 ?cast ;
         wdt:P577 ?date .
  

   # using start time and end time property to bound period
   wd:Q826477 wdt:P580 ?start .
   wd:Q826477 wdt:P582 ?end .
   
   FILTER ( ?date > ?start && ?date < ?end )
   

   # get the label
   #?season sc:name ?seasonName.
   #?episode sc:name ?episodeName .
   ?cast sc:name ?castName .
   ?film sc:name ?filmName .

}
ORDER BY DESC (?date)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q7333560'), ('filmName', 'Right Place, Right Time'), ('date', '2009-05-04T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q52664300'), ('filmName', 'November 22nd'), ('date', '2009-04-26T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q52664297'), ('filmName', 'Stealing Home'), ('date', '2009-04-12T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q52664292'), ('filmName', 'Mind Games'), ('date', '2009-03-22T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q52664289'), ('filmName', 'Officer Down'), ('date', '2009-03-15T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q52664286'), ('filmName', 'Jackals'), ('date', '2009-03-08T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q1165290'), ('filmName', 'Dadnapped'), ('date', '2009-02-16T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q52664285'), ('filmName', 'Witness Protection'), ('date', '2009-02-15T00:00:00Z')]
[('film', 'http://www.wikidata.or

In [29]:

queryString = """
SELECT DISTINCT ?cast ?castName ?filmName
WHERE {

   # using has part property to find casts
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   ?film wdt:P31 wd:Q11424 ;
         wdt:P161 ?cast ;
         wdt:P577 ?date .
  

   # using start time and end time property to bound period
   wd:Q826477 wdt:P580 ?start .
   wd:Q826477 wdt:P582 ?end .
   
   FILTER ( ?date > ?start && ?date < ?end )


   # get the label
   #?season sc:name ?seasonName.
   #?episode sc:name ?episodeName .
   ?cast sc:name ?castName .
   ?film sc:name ?filmName .

}
GROUP BY (?cast) (?castName)
#ORDER BY DESC (?filmNum)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q238501'), ('castName', 'Alex Veadov'), ('filmName', 'Drag Me to Hell')]
[('cast', 'http://www.wikidata.org/entity/Q238501'), ('castName', 'Alex Veadov'), ('filmName', 'We Own the Night')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('castName', 'Charles S. Dutton'), ('filmName', 'Against the Ropes')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('castName', 'Charles S. Dutton'), ('filmName', 'The Third Nail')]
[('cast', 'http://www.wikidata.org/entity/Q3265830'), ('castName', 'Lucille Soong'), ('filmName', 'Just like Heaven')]
[('cast', 'http://www.wikidata.org/entity/Q22704603'), ('castName', 'Lily Knight'), ('filmName', 'Changeling')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('castName', 'Charles S. Dutton'), ('filmName', 'Gothika')]
[('cast', 'http://www.wikidata.org/entity/Q459638'), ('castName', 'Stacy Edwards'), ('filmName', 'Superbad')]
[('cast', 'http://www.wikidata.org/entity/Q355163'), ('castName'

In [35]:
#Alternative with sub query

queryString = """
SELECT DISTINCT ?cast ?castName ?filmName
WHERE {
 
   ?film wdt:P31 wd:Q11424 ;
         wdt:P161 ?cast ;
         wdt:P577 ?date .
           
           { SELECT DISTINCT ?cast 
             WHERE {
           
               # using has part property to find Without a trace cast members
               wd:Q826477 wdt:P527 ?season .
               ?season wdt:P527 ?episode .
               ?episode wdt:P161 ?cast .
                  }
           }

   # using start time and end time property to bound period
   wd:Q826477 wdt:P580 ?start .
   wd:Q826477 wdt:P582 ?end .
   
   FILTER ( ?date > ?start && ?date < ?end )


   # get the label
   #?season sc:name ?seasonName.
   #?episode sc:name ?episodeName .
   ?cast sc:name ?castName .
   ?film sc:name ?filmName .

}
GROUP BY (?cast) (?castName)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q3265830'), ('castName', 'Lucille Soong'), ('filmName', 'Just like Heaven')]
[('cast', 'http://www.wikidata.org/entity/Q238501'), ('castName', 'Alex Veadov'), ('filmName', 'Drag Me to Hell')]
[('cast', 'http://www.wikidata.org/entity/Q238501'), ('castName', 'Alex Veadov'), ('filmName', 'We Own the Night')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('castName', 'Charles S. Dutton'), ('filmName', 'The Third Nail')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('castName', 'Charles S. Dutton'), ('filmName', 'Against the Ropes')]
[('cast', 'http://www.wikidata.org/entity/Q355163'), ('castName', 'Bruce Davison'), ('filmName', 'The Dead Girl')]
[('cast', 'http://www.wikidata.org/entity/Q22704603'), ('castName', 'Lily Knight'), ('filmName', 'Changeling')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('castName', 'Charles S. Dutton'), ('filmName', 'Gothika')]
[('cast', 'http://www.wikidata.org/entity/Q1125651'), ('cas

Final query for this task

In [33]:

queryString = """
SELECT DISTINCT ?cast ?castName (COUNT (DISTINCT ?film) as ?filmNum )
WHERE {
   
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   ?film wdt:P31 wd:Q11424 ;
         wdt:P161 ?cast ;
         wdt:P577 ?date .

   # using start time and end time property to bound period
   wd:Q826477 wdt:P580 ?start .
   wd:Q826477 wdt:P582 ?end .
   
   FILTER ( ?date > ?start && ?date < ?end )
   
   # get the label
   ?cast sc:name ?castName .

}
GROUP BY (?cast) (?castName)
ORDER BY DESC (?filmNum)
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q355163'), ('castName', 'Bruce Davison'), ('filmNum', '9')]
1


## Task 5
Compare Without a Trace with the tv series "Cold Case" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

In [28]:
#[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('name', 'number of seasons')]
#[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
# Using Task 3:

queryString = """
SELECT DISTINCT ?tvs ?tvsName ?castName
WHERE {

   VALUES ?tvs { wd:Q826477 wd:Q733960 }
   # using has part property
   ?tvs wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   # get the label
   #?season sc:name ?seasonName.
   #?episode sc:name ?episodeName .
   ?cast sc:name ?castName .
   ?tvs sc:name ?tvsName .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('tvs', 'http://www.wikidata.org/entity/Q733960'), ('tvsName', 'Cold Case'), ('castName', 'Justina Machado')]
[('tvs', 'http://www.wikidata.org/entity/Q733960'), ('tvsName', 'Cold Case'), ('castName', 'Shalim Ortiz')]
[('tvs', 'http://www.wikidata.org/entity/Q733960'), ('tvsName', 'Cold Case'), ('castName', 'Stephen Full')]
[('tvs', 'http://www.wikidata.org/entity/Q733960'), ('tvsName', 'Cold Case'), ('castName', 'Damon Herriman')]
[('tvs', 'http://www.wikidata.org/entity/Q733960'), ('tvsName', 'Cold Case'), ('castName', 'Paula Malcomson')]
[('tvs', 'http://www.wikidata.org/entity/Q733960'), ('tvsName', 'Cold Case'), ('castName', 'Martin Spanjers')]
[('tvs', 'http://www.wikidata.org/entity/Q733960'), ('tvsName', 'Cold Case'), ('castName', 'Kay Lenz')]
[('tvs', 'http://www.wikidata.org/entity/Q733960'), ('tvsName', 'Cold Case'), ('castName', 'Chelsea Hobbs')]
[('tvs', 'http://www.wikidata.org/entity/Q733960'), ('tvsName', 'Cold Case'), ('castName', 'Susan Walters')]
[('tvs', 'h

In [27]:
#[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('name', 'number of seasons')]
#[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
# Using Task 3:

queryString = """
SELECT DISTINCT ?tvs ?tvsName ?seasonNum ?episodeNum
WHERE {

   VALUES ?tvs { wd:Q826477 wd:Q733960 }
   # using has part property to find casts
   ?tvs wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   # finding number of seasons and episodes 
   ?tvs wdt:P2437 ?seasonNum .
   ?tvs wdt:P1113 ?episodeNum .
   
   # get the label
   #?season sc:name ?seasonName.
   #?episode sc:name ?episodeName .
   #?cast sc:name ?castName .
   ?tvs sc:name ?tvsName .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('tvs', 'http://www.wikidata.org/entity/Q733960'), ('tvsName', 'Cold Case'), ('seasonNum', '7'), ('episodeNum', '156')]
[('tvs', 'http://www.wikidata.org/entity/Q826477'), ('tvsName', 'Without a Trace'), ('seasonNum', '7'), ('episodeNum', '160')]
2


Final query for this task

In [31]:
#[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('name', 'number of seasons')]
#[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
# Using Task 3:

queryString = """
SELECT DISTINCT ?tvs ?tvsName ?seasonNum ?episodeNum (COUNT (?cast) as ?castNum )
WHERE {

   VALUES ?tvs { wd:Q826477 wd:Q733960 }
   # using has part property to find casts
   ?tvs wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   # finding number of seasons and episodes 
   ?tvs wdt:P2437 ?seasonNum .
   ?tvs wdt:P1113 ?episodeNum .
   
   # get the label
   #?season sc:name ?seasonName.
   #?episode sc:name ?episodeName .
   #?cast sc:name ?castName .
   ?tvs sc:name ?tvsName .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('tvs', 'http://www.wikidata.org/entity/Q733960'), ('tvsName', 'Cold Case'), ('seasonNum', '7'), ('episodeNum', '156'), ('castNum', '1579')]
[('tvs', 'http://www.wikidata.org/entity/Q826477'), ('tvsName', 'Without a Trace'), ('seasonNum', '7'), ('episodeNum', '160'), ('castNum', '825')]
2


## Task 6
Return the actors who are members of the cast of Without a Trace have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

In [48]:
#returns all casts of the Without a Trace
queryString = """
SELECT DISTINCT ?cast ?castName
WHERE {

   # using has part property to find casts
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   # get the label
   #?season sc:name ?seasonName.
   #?episode sc:name ?episodeName .
   ?cast sc:name ?castName .
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('cast', 'http://www.wikidata.org/entity/Q373989'), ('castName', 'David Paymer')]
[('cast', 'http://www.wikidata.org/entity/Q139341'), ('castName', 'Zach Grenier')]
[('cast', 'http://www.wikidata.org/entity/Q3265830'), ('castName', 'Lucille Soong')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('castName', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity/Q238501'), ('castName', 'Alex Veadov')]
[('cast', 'http://www.wikidata.org/entity/Q308124'), ('castName', 'Anthony LaPaglia')]
[('cast', 'http://www.wikidata.org/entity/Q1721796'), ('castName', 'Kaitlin Doubleday')]
[('cast', 'http://www.wikidata.org/entity/Q355163'), ('castName', 'Bruce Davison')]
[('cast', 'http://www.wikidata.org/entity/Q22704603'), ('castName', 'Lily Knight')]
[('cast', 'http://www.wikidata.org/entity/Q445325'), ('castName', 'Megan Gallagher')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('castName', 'Charles S. Dutton')]
[('cast', 'http://www.wikidata.org/entity/Q11256

In [69]:
#returns all properties of the casts of Without a Trace
queryString = """
SELECT DISTINCT ?pname ?oname
WHERE {

   # using has part property to find casts
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   ?cast ?p ?o .
   
   # get the label
   #?cast sc:name ?castName .
   ?p sc:name ?pname .
   ?o sc:name ?oname .
   
   #FILTER (REGEX (?pname, ".*[kK]evin.*") || (REGEX (?pname, ".*[bB]acon.*") ))
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'occupation'), ('oname', 'television actor')]
[('pname', 'occupation'), ('oname', 'film actor')]
[('pname', 'occupation'), ('oname', 'television director')]
[('pname', 'occupation'), ('oname', 'stage actor')]
[('pname', 'occupation'), ('oname', 'voice actor')]
[('pname', 'occupation'), ('oname', 'screenwriter')]
[('pname', 'occupation'), ('oname', 'actor')]
[('pname', 'occupation'), ('oname', 'character actor')]
[('pname', 'eye color'), ('oname', 'brown')]
[('pname', 'nominated for'), ('oname', 'Academy Award for Best Supporting Actress')]
[('pname', 'nominated for'), ('oname', 'Academy Award for Best Supporting Actor')]
[('pname', 'nominated for'), ('oname', 'Tony Award for Best Featured Actor in a Play')]
[('pname', 'languages spoken, written or signed'), ('oname', 'English')]
[('pname', 'ethnic group'), ('oname', 'Jewish people')]
[('pname', 'hair color'), ('oname', 'brown hair')]
[('pname', 'place of birth'), ('oname', 'Englewood')]
[('pname', 'place of birth'), 

In [53]:
# as an example: [('cast', 'http://www.wikidata.org/entity/Q373989'), ('castName', 'David Paymer')]
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {


    wd:Q373989 ?p ?o .
   
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname .
   
   
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10798782'), ('oname', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10800557'), ('oname', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pname', 'given name'), ('o', 'http://www.wikidata.org/entity/Q18057751'), ('oname', 'David')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q2059704'), ('oname', 'television director')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q230492'), ('oname', 'University of Michigan')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q2405480'), ('oname', 'voice actor')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pn

In [56]:
#[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), 
#('o', 'http://www.wikidata.org/entity/Q33999'), ('oname', 'actor')]

queryString = """
SELECT DISTINCT  ?pname  ?oname
WHERE {


    wd:Q33999 ?p ?o .
   
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname .
   
   
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'properties for this type'), ('oname', 'IMDb ID')]
[('pname', 'field of this occupation'), ('oname', 'acting')]
[('pname', 'different from'), ('oname', 'performing artist')]
[('pname', 'subclass of'), ('oname', 'performing artist')]
[('pname', "topic's main category"), ('oname', 'Category:Actors')]
[('pname', 'described by source'), ('oname', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('pname', 'has part'), ('oname', 'television actor')]
[('pname', 'has part'), ('oname', 'film actor')]
[('pname', 'described by source'), ('oname', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('pname', 'instance of'), ('oname', 'occupation')]
[('pname', 'has part'), ('oname', 'stage actor')]
[('pname', 'instance of'), ('oname', 'profession')]
[('pname', 'described by source'), ('oname', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('pname', 'instance of'), ('oname', 'occupation group according to ISCO')]
[('pname', 'patron saint'), ('oname', 'Genesius 

In [72]:
# #[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), 
#('o', 'http://www.wikidata.org/entity/Q33999'), ('oname', 'actor')]

queryString = """
SELECT DISTINCT ?actor ?actorName
WHERE {


    ?actor wdt:P106 wd:Q33999 .
   
   # get the label
   ?actor sc:name ?actorName .

   FILTER (REGEX (?actorName, ".*[kK]evin.*[Bb]acon")  )
   
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('actorName', 'Kevin Bacon')]
1


In [36]:

queryString = """
SELECT DISTINCT ?pname ?oname
WHERE {


    wd:Q3454165 ?p ?o.
   
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname.

   
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'award received'), ('oname', 'Golden Globe Award for Best Actor – Miniseries or Television Film')]
[('pname', 'sibling'), ('oname', 'Michael Bacon')]
[('pname', 'occupation'), ('oname', 'television actor')]
[('pname', 'occupation'), ('oname', 'film actor')]
[('pname', 'occupation'), ('oname', 'manufacturer')]
[('pname', 'place of birth'), ('oname', 'Philadelphia')]
[('pname', 'award received'), ('oname', 'star on Hollywood Walk of Fame')]
[('pname', 'languages spoken, written or signed'), ('oname', 'English')]
[('pname', 'genre'), ('oname', 'folk rock')]
[('pname', 'occupation'), ('oname', 'stage actor')]
[('pname', 'occupation'), ('oname', 'voice actor')]
[('pname', 'occupation'), ('oname', 'film director')]
[('pname', 'country of citizenship'), ('oname', 'United States of America')]
[('pname', 'occupation'), ('oname', 'film producer')]
[('pname', 'occupation'), ('oname', 'actor')]
[('pname', 'occupation'), ('oname', 'director')]
[('pname', 'instance of'), ('oname',

In [53]:
# From Task 1: [('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
#[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('actorName', 'Kevin Bacon')]
#[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pname', 'director')]

queryString = """
SELECT DISTINCT ?film ?filmName
WHERE {

   #all films having Kevin Bacon as a cast member or director
   ?film wdt:P161|wdt:P57 wd:Q3454165 .
   
   # get the label
   ?film sc:name ?filmName .

  
   
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('film', 'http://www.wikidata.org/entity/Q122113'), ('filmName', 'Super')]
[('film', 'http://www.wikidata.org/entity/Q463927'), ('filmName', 'Diner')]
[('film', 'http://www.wikidata.org/entity/Q636062'), ('filmName', 'Loverboy')]
[('film', 'http://www.wikidata.org/entity/Q691672'), ('filmName', 'Frost/Nixon')]
[('film', 'http://www.wikidata.org/entity/Q1917925'), ('filmName', 'My Dog Skip')]
[('film', 'http://www.wikidata.org/entity/Q5265117'), ('filmName', 'Destination Anywhere (film)')]
[('film', 'http://www.wikidata.org/entity/Q1154802'), ('filmName', 'Murder in the First')]
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('filmName', 'Friday the 13th')]
[('film', 'http://www.wikidata.org/entity/Q108946'), ('filmName', 'A Few Good Men')]
[('film', 'http://www.wikidata.org/entity/Q223596'), ('filmName', 'X-Men: First Class')]
[('film', 'http://www.wikidata.org/entity/Q339045'), ('filmName', 'The Big Picture')]
[('film', 'http://www.wikidata.org/entity/Q935173'), ('film

In [43]:
queryString = """
SELECT DISTINCT ?actor ?actorName 
WHERE {

   #all actors who perfomed a film with Kevin bacon
   ?film wdt:P161|wdt:P57 wd:Q3454165 ;
         wdt:P161 ?actor .
   
   # get the label
   ?film sc:name ?filmName .
   ?actor sc:name ?actorName .

  #Excluding Kevin Bacon
  MINUS {
              ?actor wdt:P106 wd:Q33999 . 
              FILTER (REGEX (?actorName, ".*[kK]evin.*[Bb]acon"))}
   
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q15867224'), ('actorName', "Mark O'Brien")]
[('actor', 'http://www.wikidata.org/entity/Q16185788'), ('actorName', 'Kyle Catlett')]
[('actor', 'http://www.wikidata.org/entity/Q269891'), ('actorName', 'Julianna Guill')]
[('actor', 'http://www.wikidata.org/entity/Q391359'), ('actorName', 'David Harbour')]
[('actor', 'http://www.wikidata.org/entity/Q449580'), ('actorName', 'Lynne Thigpen')]
[('actor', 'http://www.wikidata.org/entity/Q918866'), ('actorName', 'Troy Garity')]
[('actor', 'http://www.wikidata.org/entity/Q1101612'), ('actorName', 'Clint Howard')]
[('actor', 'http://www.wikidata.org/entity/Q164328'), ('actorName', 'David Koechner')]
[('actor', 'http://www.wikidata.org/entity/Q189400'), ('actorName', 'Brooke Shields')]
[('actor', 'http://www.wikidata.org/entity/Q19609853'), ('actorName', 'Eloy Casados')]
[('actor', 'http://www.wikidata.org/entity/Q198684'), ('actorName', 'Wayne Duvall')]
[('actor', 'http://www.wikidata.org/entity/

In [44]:
#We have to find all films that Kevin Bacon's co-actors played

queryString = """
SELECT DISTINCT ?filmsName ?actor ?actorName 
WHERE {

   # all actors who perfomed a film with Kevin bacon
   ?film wdt:P161|wdt:P57 wd:Q3454165 ;
         wdt:P161 ?actor .
   # all films performed by Kevin Bacon's co-actors
   ?films wdt:P161 ?actor .     
   
   # get the label
   ?film sc:name ?filmName .
   ?actor sc:name ?actorName .
   ?films sc:name ?filmsName .

  #Excluding films perfomed by Kevin Bacon
  MINUS {
      ?films wdt:P161 wd:Q3454165 .}
   
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('filmsName', 'Halt and Catch Fire'), ('actor', 'http://www.wikidata.org/entity/Q15867224'), ('actorName', "Mark O'Brien")]
[('filmsName', 'The Darkest Minds'), ('actor', 'http://www.wikidata.org/entity/Q15867224'), ('actorName', "Mark O'Brien")]
[('filmsName', 'Ready or Not'), ('actor', 'http://www.wikidata.org/entity/Q15867224'), ('actorName', "Mark O'Brien")]
[('filmsName', 'Goalie'), ('actor', 'http://www.wikidata.org/entity/Q15867224'), ('actorName', "Mark O'Brien")]
[('filmsName', 'How It Ends'), ('actor', 'http://www.wikidata.org/entity/Q15867224'), ('actorName', "Mark O'Brien")]
[('filmsName', 'State Like Sleep'), ('actor', 'http://www.wikidata.org/entity/Q15867224'), ('actorName', "Mark O'Brien")]
[('filmsName', 'Arrival'), ('actor', 'http://www.wikidata.org/entity/Q15867224'), ('actorName', "Mark O'Brien")]
[('filmsName', 'Anon'), ('actor', 'http://www.wikidata.org/entity/Q15867224'), ('actorName', "Mark O'Brien")]
[('filmsName', 'Poltergeist'), ('actor', 'http://www

In [45]:
# If an actor of the "Without a trace" performed in a film that a co-actor of Kevin Bacon also played,
#then we have a list containing "Without a trace" actors with Kevin Bacon number equals to 2

queryString = """
SELECT DISTINCT ?filmsName ?castName 
WHERE {

   #returns all casts of the Without a Trace     
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   # all actors who perfomed a film with Kevin bacon
   ?film wdt:P161|wdt:P57 wd:Q3454165 ;
         wdt:P161 ?actor .
   # all films performed by Kevin Bacon's co-actors
   ?films wdt:P161 ?actor .
   #
   ?films wdt:P161 ?cast .
   
   
   # get the label
   ?film sc:name ?filmName .
   ?actor sc:name ?actorName .
   ?films sc:name ?filmsName .
   ?cast sc:name ?castName .

  #Excluding films perfomed by Kevin Bacon
  MINUS {
      ?films wdt:P161 wd:Q3454165 .}
   
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('filmsName', 'CSI: Miami'), ('castName', 'Zach Grenier')]
[('filmsName', '24'), ('castName', 'Zach Grenier')]
[('filmsName', 'Cold Case'), ('castName', 'Zach Grenier')]
[('filmsName', "City Slickers II: The Legend of Curly's Gold"), ('castName', 'David Paymer')]
[('filmsName', 'Quiz Show'), ('castName', 'David Paymer')]
[('filmsName', 'Ghost Whisperer'), ('castName', 'David Paymer')]
[('filmsName', 'Freaky Friday'), ('castName', 'Lucille Soong')]
[('filmsName', 'Working Girl'), ('castName', 'Zach Grenier')]
[('filmsName', 'The Lesser Evil'), ('castName', 'David Paymer')]
[('filmsName', 'The Cell'), ('castName', 'Marianne Jean-Baptiste')]
[('filmsName', 'Sky High'), ('castName', 'Lucille Soong')]
[('filmsName', 'City Slickers'), ('castName', 'David Paymer')]
[('filmsName', '24, season 1'), ('castName', 'Zach Grenier')]
[('filmsName', 'Edge of Tomorrow'), ('castName', 'Marianne Jean-Baptiste')]
[('filmsName', 'Et in Arcadia Ego, Part 1'), ('castName', 'David Paymer')]
[('filmsN

Final query for this task

In [4]:
# If an actor of the "Without a trace" performed in a film that a co-actor of Kevin Bacon also played,
#then we have a list containing "Without a trace" actors with Kevin Bacon number equals to 2

queryString = """
SELECT DISTINCT ?cast ?castName 
WHERE {

   #returns all casts of the Without a Trace     
   wd:Q826477 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   # all actors who perfomed a film with Kevin bacon
   ?film wdt:P161|wdt:P57 wd:Q3454165 ;
         wdt:P161 ?actor .
   # all films performed by Kevin Bacon's co-actors
   ?films wdt:P161 ?actor .
   #
   ?films wdt:P161 ?cast .
   
   
   # get the label
   ?film sc:name ?filmName .
   ?actor sc:name ?actorName .
   ?films sc:name ?filmsName .
   ?cast sc:name ?castName .

  #Excluding films perfomed by Kevin Bacon
  MINUS {
      ?films wdt:P161 wd:Q3454165 .}
   
}

"""

print("Results")
x=run_query(queryString)



Results
[('cast', 'http://www.wikidata.org/entity/Q139341'), ('castName', 'Zach Grenier')]
[('cast', 'http://www.wikidata.org/entity/Q373989'), ('castName', 'David Paymer')]
[('cast', 'http://www.wikidata.org/entity/Q3265830'), ('castName', 'Lucille Soong')]
[('cast', 'http://www.wikidata.org/entity/Q430872'), ('castName', 'Marianne Jean-Baptiste')]
[('cast', 'http://www.wikidata.org/entity/Q238501'), ('castName', 'Alex Veadov')]
[('cast', 'http://www.wikidata.org/entity/Q308124'), ('castName', 'Anthony LaPaglia')]
[('cast', 'http://www.wikidata.org/entity/Q355163'), ('castName', 'Bruce Davison')]
[('cast', 'http://www.wikidata.org/entity/Q1125651'), ('castName', 'Thom Barry')]
[('cast', 'http://www.wikidata.org/entity/Q706513'), ('castName', 'Charles S. Dutton')]
[('cast', 'http://www.wikidata.org/entity/Q235075'), ('castName', 'Poppy Montgomery')]
[('cast', 'http://www.wikidata.org/entity/Q445325'), ('castName', 'Megan Gallagher')]
[('cast', 'http://www.wikidata.org/entity/Q22704603'

## Task 7
Consider the actors who are members of the cast of Cold Case. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

In [127]:
#returns all casts of Cold case
queryString = """
SELECT DISTINCT ?cast ?castName
WHERE {

   # using has part property to find casts
   wd:Q733960 wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   # get the label
   #?season sc:name ?seasonName.
   #?episode sc:name ?episodeName .
   ?cast sc:name ?castName .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q270382'), ('castName', 'Justina Machado')]
[('cast', 'http://www.wikidata.org/entity/Q269891'), ('castName', 'Julianna Guill')]
[('cast', 'http://www.wikidata.org/entity/Q713925'), ('castName', 'Shalim Ortiz')]
[('cast', 'http://www.wikidata.org/entity/Q128821'), ('castName', 'Stephen Full')]
[('cast', 'http://www.wikidata.org/entity/Q1093867'), ('castName', 'Damon Herriman')]
[('cast', 'http://www.wikidata.org/entity/Q433198'), ('castName', 'Paula Malcomson')]
[('cast', 'http://www.wikidata.org/entity/Q1340897'), ('castName', 'Martin Spanjers')]
[('cast', 'http://www.wikidata.org/entity/Q275476'), ('castName', 'Kay Lenz')]
[('cast', 'http://www.wikidata.org/entity/Q441617'), ('castName', 'Gabrielle Miller')]
[('cast', 'http://www.wikidata.org/entity/Q449249'), ('castName', 'Chelsea Hobbs')]
[('cast', 'http://www.wikidata.org/entity/Q457063'), ('castName', 'Susan Walters')]
[('cast', 'http://www.wikidata.org/entity/Q613378'), ('castNam

In [199]:
#returns all casts of Cold case
queryString = """
SELECT DISTINCT ?castName ?tvsName
WHERE {

   # using has part property to find casts
   wd:Q733960 wdt:P527 ?ccseason .
   ?ccseason wdt:P527 ?ccepisode .
   ?ccepisode wdt:P161 ?cast .
   
   # all tv series these actors played
   ?tvs wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   # get the label
   #?season sc:name ?seasonName.
   #?episode sc:name ?episodeName .
   ?cast sc:name ?castName .
   ?tvs sc:name ?tvsName .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('castName', 'Lois Smith'), ('tvsName', 'True Blood')]
[('castName', 'Jeremy Ratchford'), ('tvsName', 'Cold Case')]
[('castName', 'Muse Watson'), ('tvsName', 'Cold Case')]
[('castName', 'James Read'), ('tvsName', 'Cold Case')]
[('castName', 'Kay Lenz'), ('tvsName', 'Cold Case')]
[('castName', 'Chelsea Hobbs'), ('tvsName', 'Cold Case')]
[('castName', 'Shalim Ortiz'), ('tvsName', 'Cold Case')]
[('castName', 'Stephen Full'), ('tvsName', 'Cold Case')]
[('castName', 'Johnathon Schaech'), ('tvsName', 'Cold Case')]
[('castName', 'Damon Herriman'), ('tvsName', 'Cold Case')]
[('castName', 'Jeffrey Combs'), ('tvsName', 'Cold Case')]
[('castName', 'Chaney Kley'), ('tvsName', 'Cold Case')]
[('castName', 'Lois Smith'), ('tvsName', 'Cold Case')]
[('castName', 'Bree Turner'), ('tvsName', 'Cold Case')]
[('castName', 'Susan Walters'), ('tvsName', 'Cold Case')]
[('castName', 'Julianna Guill'), ('tvsName', 'Cold Case')]
[('castName', 'Gabrielle Miller'), ('tvsName', 'Cold Case')]
[('castName', '

In [133]:
#From Task 1: [('obj', 'http://www.wikidata.org/entity/Q5398426'), ('objname', 'television series')]

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {

   # finding tv series properties
   wd:Q5398426 ?p ?o .
   
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname .
   
   
   FILTER (REGEX (?oname, ".*[Aa]ward.*") )
 
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P166'), ('oname', 'award received')]
1


In [194]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('o', 'http://www.wikidata.org/entity/P166'), ('oname', 'award received')]

queryString = """
SELECT DISTINCT ?tvsName ?awardName
WHERE {

   # using has part property to find casts
   wd:Q733960 wdt:P527 ?ccseason .
   ?ccseason wdt:P527 ?ccepisode .
   ?ccepisode wdt:P161 ?cast .
   
   # all tv series these actors played
   ?tvs wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   #checking if any series won an award
   ?tvs wdt:P166 ?award .
   
   # get the label
   ?cast sc:name ?castName .
   ?tvs sc:name ?tvsName .
   ?award sc:name ?awardName .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('tvsName', 'True Blood'), ('awardName', 'Genesis Award')]
[('tvsName', 'True Blood'), ('awardName', 'Golden Globe Award for Best Actress – Television Series Drama')]
[('tvsName', 'True Blood'), ('awardName', 'Satellite Award for Best Actress – Television Series Drama')]
[('tvsName', 'True Blood'), ('awardName', 'Satellite Award for Best Supporting Actor – Series, Miniseries or Television Film')]
[('tvsName', 'True Blood'), ('awardName', 'TCA Award for Outstanding New Program')]
[('tvsName', 'True Blood'), ('awardName', 'Screen Actors Guild Award for Outstanding Performance by a Stunt Ensemble in a Television Series')]
[('tvsName', 'True Blood'), ('awardName', 'Satellite Award for Best Cast – Television Series')]
[('tvsName', 'How I Met Your Mother'), ('awardName', 'Streamy Awards')]
[('tvsName', 'How I Met Your Mother'), ('awardName', "Critics' Choice Television Award for Best Supporting Actor in a Comedy Series")]
[('tvsName', 'How I Met Your Mother'), ('awardName', 'Primeti

Final query for this task

In [23]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('o', 'http://www.wikidata.org/entity/P166'), ('oname', 'award received')]

queryString = """
SELECT DISTINCT ?tvs ?tvsName ( COUNT(?award) as ?awards )
WHERE {


   # using has part property to find casts
   wd:Q733960 wdt:P527 ?ccseason .
   ?ccseason wdt:P527 ?ccepisode .
   ?ccepisode wdt:P161 ?cast .
   
   # all tv series these actors played
   ?tvs wdt:P527 ?season .
   ?season wdt:P527 ?episode .
   ?episode wdt:P161 ?cast .
   
   #checking if any series won an award
   ?tvs wdt:P166 ?award .
   
   # get the label
   ?cast sc:name ?castName .
   ?tvs sc:name ?tvsName .
   ?award sc:name ?awardName .
}
GROUP BY (?tvs) (?tvsName)
HAVING (COUNT(?award) > 2)
ORDER BY (?awards)
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('tvs', 'http://www.wikidata.org/entity/Q23577'), ('tvsName', 'Dexter'), ('awards', '4')]
[('tvs', 'http://www.wikidata.org/entity/Q183513'), ('tvsName', 'Buffy the Vampire Slayer'), ('awards', '4')]
[('tvs', 'http://www.wikidata.org/entity/Q4525'), ('tvsName', 'NCIS'), ('awards', '4')]
[('tvs', 'http://www.wikidata.org/entity/Q210146'), ('tvsName', 'Babylon 5'), ('awards', '8')]
[('tvs', 'http://www.wikidata.org/entity/Q8539'), ('tvsName', 'The Big Bang Theory'), ('awards', '10')]
[('tvs', 'http://www.wikidata.org/entity/Q34316'), ('tvsName', 'Doctor Who'), ('awards', '12')]
[('tvs', 'http://www.wikidata.org/entity/Q2744'), ('tvsName', 'The X-Files'), ('awards', '16')]
[('tvs', 'http://www.wikidata.org/entity/Q2085'), ('tvsName', 'Twin Peaks'), ('awards', '18')]
[('tvs', 'http://www.wikidata.org/entity/Q21296543'), ('tvsName', 'Star Trek: Discovery'), ('awards', '18')]
[('tvs', 'http://www.wikidata.org/entity/Q558112'), ('tvsName', 'Black Mirror'), ('awards', '24')]
[('tvs', 